In [9]:
%cd C:\Users\yukir\Documents\Monicas_workspace\train_diff\workspace

C:\Users\yukir\Documents\Monicas_workspace\train_diff\workspace


In [2]:
import cv2

### Find Starting Point & listing_img

In [27]:
cap = cv2.VideoCapture("C:\\Users\\yukir\\Documents\\Monicas_workspace\\train_diff\\org\\testing_vid\\basic.mp4")

#재생할 파일의 넓이 얻기
width = 710

#재생할 파일의 높이 얻기
height = 1080
#재생할 파일의 프레임 레이트 얻기
fps = 30

print('width {0}, height {1}, fps {2}'.format(width, height, fps))

#저장할 비디오 코덱
fourcc = cv2.VideoWriter_fourcc(*'XVID')
#저장할 파일 이름
filename = 'detection_result_test.mp4'
out = cv2.VideoWriter(filename, fourcc, fps, (int(width), int(height)))

start_basic_list = []
basic_img_list = []
frame_num = 0

ret, back = cap.read()
back = cv2.GaussianBlur(back, (0, 0), 1.0)
back = back[0:1080, 610:1320]
# back = back[400:600, 20:680]
back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)

# 가우시안 블러로 노이즈 제거 (모폴로지, 열기, 닫기 연산도 가능)
back = cv2.GaussianBlur(back, (0, 0), 1.0)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    frame = frame[0:1080, 610:1320]
    # frame = frame[400:600, 20:680]
    frame_org = frame.copy()

    frame_num +=1 
    # 현재 프레임 영상 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 노이즈 제거
    gray = cv2.GaussianBlur(gray, (0, 0), 1.0)
    
    # 차영상 구하기 $ 이진화
    # absdiff는 차 영상에 절대값
    diff = cv2.absdiff(gray, back)
    # gray_mean() or 밝기정도 지정 가능
    ####!!!!!back.mean()으로 하면 smoke까지 가능?
    _, diff = cv2.threshold(diff, 70, 255, cv2.THRESH_BINARY)
    
    # 레이브링을 이용하여 바운딩 박스 표시
    cnt, _, stats, _ = cv2.connectedComponentsWithStats(diff)
    
    #contour
#     dilated = cv2.dilate(diff, None, iteration=3)

    contours, _ = cv2.findContours(diff, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for i in range(1, cnt):
        x, y, w, h, s = stats[i]
        
        if s < 100:
            continue

       
        start_basic_list.append(frame_num)
        basic_img_list.append(frame)
        out.write(frame)

    cv2.imshow('frame', frame)
    # out.write(frame)

    if cv2.waitKey(30) == 27:
        break
    

cap.release()
out.release()
cv2.destroyAllWindows()

width 710, height 1080, fps 30


In [26]:
print(basic_img_list[0].shape)

(1080, 710, 3)


In [21]:
width = 660

#재생할 파일의 높이 얻기
height = 200
#재생할 파일의 프레임 레이트 얻기
fps = 30

print('width {0}, height {1}, fps {2}'.format(width, height, fps))

#저장할 비디오 코덱
fourcc = cv2.VideoWriter_fourcc(*'XVID')
#저장할 파일 이름
filename = 'detection_result_test.mp4'
out = cv2.VideoWriter(filename, fourcc, fps, (int(width), int(height)))

for i in basic_img_list:
    out.write(i)

out.release()

width 660, height 200, fps 30


In [33]:
save_path = 'C:\\Users\\yukir\Documents\\Monicas_workspace\\train_diff\\dataset\\data_img\\images\\'


for num, i in enumerate(basic_img_list):
    number = num*55
    try:
        img = basic_img_list[number]
        cv2.imwrite(save_path+'basic_{}.jpg'.format(num), img)
    except:
        pass

In [17]:
%cd C:\Users\yukir\Documents\Monicas_workspace\train_diff\workspace\yolov7-custom

C:\Users\yukir\Documents\Monicas_workspace\train_diff\workspace\yolov7-custom


In [30]:
!python detect.py --weights best_v1.pt --conf 0.6 --source C:\Users\yukir\Documents\Monicas_workspace\train_diff\workspace\detection_result_test.mp4

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.6, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='C:\\Users\\yukir\\Documents\\Monicas_workspace\\train_diff\\workspace\\detection_result_test.mp4', update=False, view_img=False, weights=['best_v1.pt'])
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

video 1/1 (1/2510) C:\Users\yukir\Documents\Monicas_workspace\train_diff\workspace\detection_result_test.mp4: Done. (10.0ms) Inference, (1.0ms) NMS
video 1/1 (2/2510) C:\Users\yukir\Documents\Monicas_workspace\train_diff\workspace\detection_result_test.mp4: Done. (11.0ms) Inference, (0.0ms) NMS
video 1/1 (3/2510) C:\Users\yukir\Documents\Monicas_workspace\train_diff\workspace\detection_result_test.mp4: Done. (10

YOLOR  2022-10-8 torch 1.12.0+cu116 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.5625MB)

c:\Users\yukir\anaconda3\envs\yolov7\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS
